In [1]:
# for correct relative imports
import sys; sys.path.append("../var_es_dgm")

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from diffusers import DDPMScheduler
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

from sklearn.preprocessing import StandardScaler

from var_es_dgm import TimeGrad
from var_es_dgm.basic_models.deepvar import DeepVaR
from var_es_dgm.basic_models import HistoricalSimulation, VarCov
from var_es_dgm.stat_tests import generate_report
from var_es_dgm.utils import seed_everything, compute_individual_returns, compute_portfolio_returns, \
    estimate_var_es_torch_multivariate, pick_device

In [ ]:
device = pick_device()

In [3]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

In [4]:
DATA_FOLDER = "../../data/"
df = pd.read_csv(DATA_FOLDER + "complete_stocks.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
len(df["Ticker"].unique())

89

In [6]:
res_timeGrad = list()
res_hist = list()
res_varcov = list()
res_deepvar = list()
alpha = 0.05

In [7]:
RANDOM_STATE = 12

In [8]:
for i in range(5):
    # one complete cycle
    seed_everything(RANDOM_STATE + i)
    n_stocks = 10
    tickers = np.random.choice(df["Ticker"].unique(), n_stocks, replace=False)
    weights = 1/n_stocks
    print("Portfolio = {0}".format(" + ".join([f"{weights} * {i}"for i in tickers])))
    df_copy = df.loc[df["Ticker"].isin(tickers)].copy(deep=True)

    df_returns = compute_individual_returns(df_copy)
    df_returns = compute_portfolio_returns(df_returns)

    return_cols = [i for i in df_returns.columns if (i.startswith("Return_") and i != "Return_Target")]
    multivariate_returns = df_returns[return_cols]
    multivariate_target = df_returns["Return_Target"]

    multivariate_target = multivariate_target.values[1:]
    train_size = df_returns[df_returns.Date <= "2022-06-01"].index[-1] + 1
    test_size = len(multivariate_target) - train_size
    train = multivariate_returns.values[1:train_size]

    ss = StandardScaler()
    train_scaled = torch.tensor(ss.fit_transform(train), dtype=torch.float32)

    seed_everything(RANDOM_STATE)
    context_size = 90
    num_train_samples = 3000
    train_data = torch.zeros(num_train_samples, context_size, train_scaled.shape[1])
    train_target = torch.zeros(num_train_samples, 1, train_scaled.shape[1])
    train_idx = np.random.choice(np.arange(context_size, train_scaled.shape[0]), num_train_samples, replace=False)

    for i in tqdm(range(num_train_samples)):
        idx = train_idx[i]
        train_context = train_scaled[idx-context_size:idx]
        target_obs = train_scaled[idx]
        train_data[i] = train_context
        train_target[i] = target_obs
    
    # Create DataLoader for ease of torch training
    train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=128, shuffle=False)


    temp = torch.tensor(ss.transform(multivariate_returns.values[1:]))
    test_data_context = torch.zeros(test_size, context_size, temp.shape[1])
    test_data_real = torch.zeros(test_size, 1, 1)
    for i in range(test_size):
        idx = i + train_size
        test_data_context[i] = temp[idx-context_size:idx]
        test_data_real[i] = multivariate_target[idx]

    seed_everything(RANDOM_STATE)
    sheduler = DDPMScheduler(num_train_timesteps=12, beta_end=0.022429104089340533, clip_sample=False)
    model = TimeGrad(train.shape[-1], train.shape[-1], hidden_size=50, num_layers=2, scheduler=sheduler, num_inference_steps=12)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0030588361272846074)
    n_epochs = 96
    model.to(device)

    model.fit(train_loader, optimizer, n_epochs, str(device))


    seed_everything(RANDOM_STATE)
    VaR_TimeGrad = torch.zeros(test_data_real.shape[0])
    ES_TimeGrad = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        
        # compute correlation matrix
        pho = torch.corrcoef(torch.squeeze(test).T).to(torch.double)
        
        VaR_TimeGrad[i], ES_TimeGrad[i] = estimate_var_es_torch_multivariate(model, test, ss, pho, alpha=alpha, n_samples=500, device=device)

    res_timeGrad.append(generate_report(test_data_real.flatten(), VaR_TimeGrad, ES_TimeGrad, alpha=alpha))

    # DeepVaR model
    seed_everything(RANDOM_STATE)
    model = DeepVaR(target_dim=multivariate_returns.shape[1], input_size=multivariate_returns.shape[1], hidden_size=64, num_layers=2)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    n_epochs = 90
    model.to(device)

    model.fit(train_loader, optimizer, n_epochs, device)

    VaR_DeepVaR = torch.zeros(test_data_real.shape[0])
    ES_DeepVaR = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_DeepVaR[i], ES_DeepVaR[i] = estimate_var_es_torch_multivariate(model, test, ss, pho, alpha=alpha, n_samples=500, device=device)

    res_deepvar.append(generate_report(test_data_real.flatten(), VaR_DeepVaR, ES_DeepVaR, alpha=alpha))

    hist_sim = HistoricalSimulation(alpha=alpha)

    VaR_histSim = torch.zeros(test_data_real.shape[0])
    ES_histSim = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_histSim[i], ES_histSim[i] = hist_sim.predict(test_data_context[[i]], scaler=ss)
    
    res_hist.append(generate_report(test_data_real.flatten(), VaR_histSim, ES_histSim, alpha=alpha))
    
    var_cov = VarCov(alpha=alpha)
    
    VaR_varCov = torch.zeros(test_data_real.shape[0])
    ES_varCov = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_varCov[i], ES_varCov[i] = var_cov.predict(test_data_context[[i]], scaler=ss)
    
    res_varcov.append(generate_report(test_data_real.flatten(), VaR_varCov, ES_varCov, alpha=alpha))

    print(res_timeGrad[-1], res_deepvar[-1], res_hist[-1], res_varcov[-1])

Portfolio = 0.1 * INTC + 0.1 * DHR + 0.1 * COP + 0.1 * MRK + 0.1 * HON + 0.1 * T + 0.1 * MSFT + 0.1 * AXP + 0.1 * PEP + 0.1 * SYK


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

Epochs:   0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

{'Kupicks POF': 0.5086437093214728, 'Haas TBF': 0.06669736650413338, 'Christoffersen Test': 0.5304092795638204, 'Kupicks Test': 0.5086437093214728, 'Acerbi Szekely 1': 2.712733030319214, 'Acerbi Szekely 2': 0.005822785664349794} {'Kupicks POF': 0.009515381736260306, 'Haas TBF': 0.004674007496692546, 'Christoffersen Test': 0.42499904223958085, 'Kupicks Test': 0.009515381736260306, 'Acerbi Szekely 1': 1.7800683975219727, 'Acerbi Szekely 2': 0.007192195393145084} {'Kupicks POF': 0.11478649504904415, 'Haas TBF': 0.03856108825211334, 'Christoffersen Test': 0.451493118695405, 'Kupicks Test': 0.11478649504904415, 'Acerbi Szekely 1': 2.386465549468994, 'Acerbi Szekely 2': 0.008135678246617317} {'Kupicks POF': 0.8527157106323404, 'Haas TBF': 0.12667139932019633, 'Christoffersen Test': 0.5098255905975888, 'Kupicks Test': 0.8527157106323404, 'Acerbi Szekely 1': 2.077404499053955, 'Acerbi Szekely 2': 0.00498367240652442}


In [9]:
print(np.array([list(i.values()) for i in res_timeGrad]).mean(axis=0))
print(np.array([list(i.values()) for i in res_deepvar]).mean(axis=0))
print(np.array([list(i.values()) for i in res_hist]).mean(axis=0))
print(np.array([list(i.values()) for i in res_varcov]).mean(axis=0))

[0.48472364 0.07868907 0.53222904 0.48472364 2.70502515 0.0059692 ]
[0.34594714 0.01684031 0.45935732 0.34594714 1.80093093 0.00611885]
[0.45848716 0.03499475 0.47515655 0.45848716 2.40126729 0.00716972]
[0.59423254 0.1037123  0.52292489 0.59423254 2.0204572  0.00444999]
